In [17]:
# --- 1. SETUP ---
from google.colab import drive
import joblib
import re
import os
import pandas as pd

# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Paths
PROJECT_PATH = "/content/drive/My Drive/Smart Complaint Prioritizer"
MODEL_PATH = f"{PROJECT_PATH}/models/priority_model.pkl"
VECTORIZER_PATH = f"{PROJECT_PATH}/models/tfidf_vectorizer.pkl"

# --- 2. LOAD MODELS ---
print("Loading Pro-Version Models...", end=" ")
try:
    model = joblib.load(MODEL_PATH)
    tfidf = joblib.load(VECTORIZER_PATH)
    print("✅ Done.")
except FileNotFoundError:
    print("❌ Error: Models not found.")

# --- 3. THE SMART PREDICTOR ---
def predict_priority(text):
    # 1. Clean Text
    clean_text = str(text).lower()
    clean_text = re.sub(r'[^a-z\s]', '', clean_text)

    # 2. Get AI Prediction
    vec = tfidf.transform([clean_text])
    pred = model.predict(vec)[0]
    prob = model.predict_proba(vec).max()

    # 3. SAFETY NET (The Logic Fix)
    # Even with the smarter model, we keep this for 100% safety on critical threats.
    critical_keywords = [
        'stole', 'stolen', 'theft', 'robbery', 'sue', 'lawsuit', 'fraud', 'illegal',
        'killed', 'kill', 'died', 'death', 'threat', 'danger', 'police', 'arrest', 'prison'
    ]

    # Check if any critical word is in the text
    found_keyword = None
    for word in critical_keywords:
        if word in clean_text.split(): # Exact word match
            found_keyword = word
            break

    # 4. Apply Override
    final_priority = pred
    note = ""

    if found_keyword and pred != "High":
        final_priority = "High"
        note = f"(Override: Found '{found_keyword}')"
        prob = 1.0 # Force max confidence on override

    return final_priority, prob, note

# --- 4. RUN ---
print("\n--- 🏦 SMART COMPLAINT PRIORITIZER (PRO) ---")
print("Type 'quit' to exit.\n")

while True:
    user_input = input("Enter a complaint: ")
    if user_input.lower() in ['quit', 'exit']:
        break

    # Get Result
    priority, confidence, note = predict_priority(user_input)

    # Print Result
    print(f"👉 Priority: {priority.upper()} {note}")
    print(f"📊 Confidence: {confidence*100:.1f}%")
    print("-" * 30)

Loading Pro-Version Models... ✅ Done.

--- 🏦 SMART COMPLAINT PRIORITIZER (PRO) ---
Type 'quit' to exit.

Enter a complaint: someone killed my friend
👉 Priority: HIGH (Override: Found 'killed')
📊 Confidence: 100.0%
------------------------------
Enter a complaint: I am very frustrated with the delays in my account setup.
👉 Priority: HIGH 
📊 Confidence: 87.4%
------------------------------
Enter a complaint: if you dont stop calling me. I will call police
👉 Priority: HIGH (Override: Found 'police')
📊 Confidence: 100.0%
------------------------------
Enter a complaint: I woke up this morning and saw a transaction of $5,000 that I never made. My rent is due tomorrow!
👉 Priority: LOW 
📊 Confidence: 63.6%
------------------------------
Enter a complaint: You guys are criminals. This whole bank is a scam operation.
👉 Priority: HIGH 
📊 Confidence: 100.0%
------------------------------
Enter a complaint: I noticed a duplicate charge of $25.00 on my statement for October.
👉 Priority: MEDIUM 
📊 C